In [1]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input


Using TensorFlow backend.


In [2]:
# read the CSV into memory
prices = []
image_paths = []

data_path = "../datasets/bikes_im/"
with open("../datasets/bikes_filtered.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = row[2]
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(int(msrp))

train_indices = np.load("bikes_train_indices.npy")
test_indices = np.load("bikes_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

(19658,)
(2185,)


In [3]:
# Take a data point and run it through the neural network
# Return the predicted value and calculate the MSE
def evaluate(model, index):
    msrp = prices[index]
    path = image_paths[index]
    img = image.load_img(path, target_size=(224, 224))
    data = np.expand_dims(image.img_to_array(img), axis=0)
    prediction = model.predict(data, msrp)
    #prediction = new_model.predict(data, np.expand_dims(np.array(msrp), axis=0))
    #print("Bike index: " + str(index))
    #print("Actual price: " + str(msrp))
    #print("Predicted price: " + str(prediction))
    mse = (prediction-msrp)**2
    return mse

def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 1))
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 1))
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)                
                Y[i] = prices[index]
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

In [4]:
# Hyperparameters

hp_dropout = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

#RMSprop
hp_lr = [0.01] * 7
hp_rho = [0.9] * 7
hp_epsilon = [1e-07] * 7
hp_decay = [0.0] * 7

# Number of hidden units
hp_hidden = [256] * 7

# Minibatch size
hp_mbsize = [64] * 7

In [ ]:
# Each index in the list corresponds to a HP setting
# E.G. setting = 1 will select dropout = 0.01, lr = 0.01, rho = 0.9 ...
# TODO: vary the settings, right now I only vary dropout and clamp all the others to a single value

for setting in range(len(hp_dropout)):
    # build the VGG16 network
    input_tensor = Input(shape=(224,224,3))
    model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)
    
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=(model.output_shape[1:])))


    # Output layer
    # We do random weight intialization
    top_model.add(Dropout(hp_dropout[setting]))
    top_model.add(Dense(hp_hidden[setting], activation='relu', kernel_initializer='glorot_uniform'))
    top_model.add(Dense(1, activation='linear', name='output', kernel_initializer='glorot_uniform'))
    
    # add the model on top of the convolutional base
    new_model = Model(inputs= model.input, outputs = top_model(model.output))

    # set the first 19 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in new_model.layers[:19]:
        layer.trainable = False

    # RMSprop
    new_model.compile(loss='mean_squared_error',
                      optimizer=optimizers.RMSprop(lr=hp_lr[setting], rho=hp_rho[setting], epsilon=hp_epsilon[setting], decay=hp_decay[setting]))
    
    epochs = 30
    minibatch_size = hp_mbsize[setting]

    train_steps = math.ceil(len(train_indices) / minibatch_size)
    test_steps = math.ceil(len(test_indices) / minibatch_size)

    # fine-tune the model
    history = new_model.fit_generator(
        image_generator(train_indices, minibatch_size),
        steps_per_epoch=train_steps,
        epochs=epochs,
        validation_data=image_generator(test_indices, minibatch_size),
        nb_val_samples=test_steps)
    
    error = []
    for i in range(21843):
        error.append(evaluate(new_model,i))
    print(np.mean(error))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()
    
    print("==========")